In [1]:
import numpy as np
import scipy as sp
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.metrics import accuracy_score

In [2]:
df_bt = pd.read_csv('blocks_timestamps.csv') # Timestamps for each block
df_om = pd.read_csv('ordermatches_all.csv')  # Order details for each order (block)
df_pp = pd.read_csv('pepecash_prices.csv')   # USD value of pepecash over time (timeblock)
df_xp = pd.read_csv('xcp_prices.csv')        # USD value of xcp over time (timeblock)

In [3]:
df_bt

,Block,Timestamp
0,428925,1473387721
1,428926,1473388413
2,428927,1473391665
3,428928,1473391811
4,428929,1473393165
...,...,...
78260,507185,1517534266
78261,507186,1517535116
78262,507187,1517535355
78263,507188,1517535610


In [4]:
df_om

,Block,ForwardAsset,ForwardQuantity,BackwardAsset,BackwardQuantity
0,428940,RAREPEPE,1.00,XCP,2.00
1,429221,SHITCOINCARD,230.00,XCP,13.80
2,429221,RAREPEPE,10.00,XCP,15.00
3,429228,SHITCOINCARD,3.00,XCP,0.18
4,429228,RAREPEPE,1.00,XCP,1.50
...,...,...,...,...,...
26033,507444,PEPECASH,600.00,KEKEVOLUTION,2.00
26034,507444,PIRATPEPE,5.00,PEPECASH,6000.00
26035,507444,SISTNCHAPEPE,3.00,PEPECASH,2025.00
26036,507447,XCP,0.75,SISTNCHAPEPE,1.00


In [5]:
df_pp

,Timestamp,Price
0,1517558400,0.052494
1,1517472000,0.041177
2,1517385600,0.062411
3,1517299200,0.064185
4,1517212800,0.071138
...,...,...
486,1475478000,0.000069
487,1475391600,0.000069
488,1475305200,0.000037
489,1475218800,0.000084


In [6]:
df_xp

,Timestamp,Price
0,1517558400,32.75
1,1517472000,33.52
2,1517385600,43.16
3,1517299200,44.83
4,1517212800,53.25
...,...,...
515,1473058800,3.55
516,1472972400,3.48
517,1472886000,3.57
518,1472799600,3.51


In [7]:
# Need to throw out all data that isn't xcp or pepecash value
# JK no we don't
# df_om.drop(df_om[df_om.ForwardAsset != ('PEPECASH' or 'XCP')].index, inplace=True)

In [8]:
#df_om_merge = pd.concat([df_om, df_bt], axis=1, join='inner')
df_om_merge = pd.merge(df_om, df_bt, on='Block', how='inner')


In [9]:
#df_om.insert
#df_om_merge = pd.merge(df_om, df_bt)
#df_om_merge = pd.concat([df_om, df_bt], on='Block', axis=1)
df_om_merge

,Block,ForwardAsset,ForwardQuantity,BackwardAsset,BackwardQuantity,Timestamp
0,428940,RAREPEPE,1.0,XCP,2.00,1473403689
1,429221,SHITCOINCARD,230.0,XCP,13.80,1473561677
2,429221,RAREPEPE,10.0,XCP,15.00,1473561677
3,429228,SHITCOINCARD,3.0,XCP,0.18,1473567368
4,429228,RAREPEPE,1.0,XCP,1.50,1473567368
...,...,...,...,...,...,...
25969,507166,PEPECASH,999.0,SISTNCHAPEPE,3.00,1517525368
25970,507184,DONTBEMENACE,1.0,PEPECASH,67.00,1517533848
25971,507187,BCCPEPE,1.0,PEPECASH,150.00,1517535355
25972,507187,SISTNCHAPEPE,1.0,PEPECASH,333.00,1517535355


In [10]:
df_om_merge.columns

Index(['Block', 'ForwardAsset', 'ForwardQuantity', 'BackwardAsset',
       'BackwardQuantity', 'Timestamp'],
      dtype='object')

In [11]:
#for row in df_om_merge:
df_om_merge2 = df_om_merge
for x, y in df_om_merge2.iterrows():
    check = False
    i = 0
    if(df_om_merge2.loc[x, 'BackwardAsset'] == 'XCP'):
        while check != True or i<519:
            if(df_om_merge2.loc[x, 'Timestamp'] < df_xp.loc[i, 'Timestamp'] and df_om_merge2.loc[x, 'Timestamp'] >= df_xp.loc[i+1, 'Timestamp']):
                check = True
                df_om_merge2.BackwardQuantity[x] = df_om_merge2.BackwardQuantity[x] * df_xp.Price[i+1]
                df_om_merge2.BackwardAsset[x] = 'USD'
            i=i+1
        i=0
    check = False
    if(df_om_merge2.loc[x, 'ForwardAsset'] == 'XCP'):
        while check != True or i<519:
            if(df_om_merge2.loc[x, 'Timestamp'] < df_xp.loc[i, 'Timestamp'] and df_om_merge2.loc[x, 'Timestamp'] >= df_xp.loc[i+1, 'Timestamp']):
                check = True
                df_om_merge2.ForwardQuantity[x] = df_om_merge2.ForwardQuantity[x] * df_xp.Price[i+1]
                df_om_merge2.ForwardAsset[x] = 'USD'
            i=i+1
        i=0
        
    # PEPECASH Conversion
    check = False
    if(df_om_merge2.loc[x, 'BackwardAsset'] == 'PEPECASH'):
        while check != True or i<490:
            if(df_om_merge2.loc[x, 'Timestamp'] < df_pp.loc[i, 'Timestamp'] and df_om_merge2.loc[x, 'Timestamp'] >= df_pp.loc[i+1, 'Timestamp']):
                check = True
                df_om_merge2.BackwardQuantity[x] = df_om_merge2.BackwardQuantity[x] * df_pp.Price[i+1]
                df_om_merge2.BackwardAsset[x] = 'USD'
            i=i+1
        i=0
    check = False
    if(df_om_merge2.loc[x, 'ForwardAsset'] == 'PEPECASH'):
        while check != True or i<490:
            if(df_om_merge2.loc[x, 'Timestamp'] < df_pp.loc[i, 'Timestamp'] and df_om_merge2.loc[x, 'Timestamp'] >= df_pp.loc[i+1, 'Timestamp']):
                check = True
                df_om_merge2.ForwardQuantity[x] = df_om_merge2.ForwardQuantity[x] * df_pp.Price[i+1]
                df_om_merge2.ForwardAsset[x] = 'USD'
            i=i+1
           
        
        #for label, row in df_xp.iterrows():
                #print(label)
                #print(row)
                #if(ts >= df_xp.Timestamp[x] and ts < df_xp.Timestamp[x+1]):
                    #print(x)
                    #df_om_merge.BackwardQuantity[i] = df_xp.Price[x] * df_om_merge.BackwardQuantity[i]
                    #df_om_merge.BackwardAsset = 'USD'
                
#print(df_om_merge.Timestamp)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versu

In [12]:
# This dataset now has the USD equivalent of the XCP values and PEPECASH
df_om_merge2

,Block,ForwardAsset,ForwardQuantity,BackwardAsset,BackwardQuantity,Timestamp
0,428940,RAREPEPE,1.000000,USD,6.660000,1473403689
1,429221,SHITCOINCARD,230.000000,USD,44.988000,1473561677
2,429221,RAREPEPE,10.000000,USD,48.900000,1473561677
3,429228,SHITCOINCARD,3.000000,USD,0.586800,1473567368
4,429228,RAREPEPE,1.000000,USD,4.890000,1473567368
...,...,...,...,...,...,...
25969,507166,USD,41.135823,SISTNCHAPEPE,3.000000,1517525368
25970,507184,DONTBEMENACE,1.000000,USD,2.758859,1517533848
25971,507187,BCCPEPE,1.000000,USD,6.176550,1517535355
25972,507187,SISTNCHAPEPE,1.000000,USD,13.711941,1517535355


In [13]:
df_om_merge2.to_csv(r'C:\Programming\cmpe188\rarepepes\ordermatches_USD.csv')